In [ ]:
%rm -rf /content/__MACOSX
%rm -rf /content/data
%mkdir /content/data

!unzip -qq /content/drive/MyDrive/captcha/debug-10k.zip
%mv /content/debug-10k /content/data/train
!echo "Done train Data"

!unzip -qq /content/drive/MyDrive/captcha/debug-2k.zip
%mv /content/debug-2k /content/data/validation
!echo "Done validation Data"

%rm -rf /content/__MACOSX

Done train Data
Done validation Data


Mount data từ Google Drive và giải nén vào folder đích

Load dataset vào bộ nhớ để chia thành 2 tập train và validation

In [ ]:
import tensorflow as tf
from tensorflow.keras import preprocessing

BATCH_SIZE = 128
COLOR_MODE = "rgba"
IMG_HEIGHT = 50
IMG_WIDTH = 180

train_ds = preprocessing.image_dataset_from_directory(
  directory="data/train",
  label_mode="categorical",
  seed=123,
  color_mode=COLOR_MODE,
  image_size=(IMG_HEIGHT, IMG_WIDTH),
  batch_size=BATCH_SIZE)

validation_ds = preprocessing.image_dataset_from_directory(
  directory="data/validation",
  label_mode="categorical",
  seed=123,
  color_mode=COLOR_MODE,
  image_size=(IMG_HEIGHT, IMG_WIDTH),
  batch_size=BATCH_SIZE)

# Configure the dataset for performance
def configure_for_performance(ds):
    ds = ds.cache()
    ds = ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    return ds

train_ds = configure_for_performance(train_ds)
validation_ds = configure_for_performance(validation_ds)

Found 45786 files belonging to 23 classes.
Found 9201 files belonging to 23 classes.


In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import optimizers
from tensorflow.keras import Sequential

MODEL_OUTPUT_DIR="/content/drive/MyDrive/captcha/captcha-model"
CLASS_NO = 23
EPOCHS = 120

def get_model():
    m = Sequential([
      layers.experimental.preprocessing.Rescaling(1./255, name=""),

      layers.Conv2D(32, (3, 3), padding="same", input_shape=(IMG_HEIGHT, IMG_WIDTH, 1)),
      layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
      layers.Conv2D(64, (3, 3), padding="same"),
      layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
      layers.Conv2D(128, (3, 3), padding="same"),
      layers.Conv2D(64, (1, 1), padding="same"),
      layers.Conv2D(128, (3, 3), padding="same"),
      layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
      layers.Conv2D(256, (3, 3), padding="same"),
      layers.Conv2D(128, (1, 1), padding="same"),
      layers.Conv2D(256, (3, 3), padding="same"),
      layers.Conv2D(128, (1, 1), padding="same"),
      layers.Conv2D(256, (3, 3), padding="same"),
      layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
      layers.Conv2D(512, (3, 3), padding="same"),
      layers.Conv2D(256, (1, 1), padding="same"),
      layers.Conv2D(512, (3, 3), padding="same"),
      layers.Conv2D(256, (1, 1), padding="same"),
      layers.Conv2D(512, (3, 3), padding="same"),
      layers.Conv2D(256, (1, 1), padding="same"),
      layers.Conv2D(512, (3, 3), padding="same"),
      layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
      layers.Flatten(),
      layers.Dropout(rate=0.5),
      layers.Dense(CLASS_NO, activation="softmax", name="output"),
    ])
    m.compile(
      loss=losses.CategoricalCrossentropy(),
      metrics=['accuracy'],
      optimizer=optimizers.Adam(learning_rate=0.00001),
    )
    return m

model=get_model()
model.fit(train_ds, epochs=EPOCHS, validation_data=validation_ds, verbose=1)
model.summary()
model.save(MODEL_OUTPUT_DIR)

Epoch 1/120
358/358 [==============================] - 48s 111ms/step - loss: 3.1314 - accuracy: 0.0524 - val_loss: 3.0154 - val_accuracy: 0.0942
Epoch 2/120
358/358 [==============================] - 33s 91ms/step - loss: 2.9697 - accuracy: 0.1056 - val_loss: 2.8357 - val_accuracy: 0.1444
Epoch 3/120
358/358 [==============================] - 34s 96ms/step - loss: 2.8125 - accuracy: 0.1452 - val_loss: 2.6381 - val_accuracy: 0.1986
Epoch 4/120
358/358 [==============================] - 35s 97ms/step - loss: 2.6326 - accuracy: 0.1935 - val_loss: 2.4266 - val_accuracy: 0.2411
Epoch 5/120
358/358 [==============================] - 34s 94ms/step - loss: 2.4311 - accuracy: 0.2400 - val_loss: 2.1645 - val_accuracy: 0.3236
Epoch 6/120
358/358 [==============================] - 34s 96ms/step - loss: 2.2132 - accuracy: 0.2987 - val_loss: 1.9175 - val_accuracy: 0.3973
Epoch 7/120
358/358 [==============================] - 34s 96ms/step - loss: 2.0099 - accuracy: 0.3540 - val_loss: 1.6799 - val_a